# Neste notebook vamos definir as funçoes de filtragem e visualizaçao dos dados a partir das classes definidas na bilbioteca GeoPandas
[Mapping and Plotting Tools](https://geopandas.org/docs/user_guide/mapping.html)

- Com as funçoes definidas e testadas neste notebook, criaremos um arquivo .py no diretorio .soureces/ para armazena-las, possibilitando assim, que possamos chama-las com a funçao import.

In [69]:
import geopandas as gpd

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable  # Tutorial GeoPandas Mapping and Pltting Tools

%matplotlib widget

In [2]:
# Origem do banco de dados
gdb = '/home/ggrl/geodatabase/'

## Importando um vetor do banco de dados criado no notebook passado.
- Escolha de camada e mapa
    - Caso nao tenha o mapa selecionado retorne lista de mapas disponiveis para aquela escala.
    - Caso nao escolha um mapa retorne todos os vetores daquela camada.

In [3]:
# IMPORTADOR DE LITOLOGIAS POR ESCALA --------------------------------------------------------------------------#
def importar_geologico(camada, mapa=False):
    lito =  gpd.read_file(gdb+'geodatabase.gpkg',
                        driver= 'GPKG',
                        layer= camada)
    if mapa:
        folha = lito[lito.MAPA == 'Carta geológica da folha '+mapa]
        if folha.empty:
            print(f"O mapa escolhido nao est'a presente na coluna MAPA da camada veotiral.")
            print(f"# -- Lista de mapas: {list(lito.MAPA.unique())}")

            return(lito)

        else:    
            return(folha)
    else:
        return(lito)

In [4]:
# Definindo uma funçao que importa as escaladas 
def plot_base(gdf,atributo=None,camada=None,mapa=None):
    litologia = importar_geologico(camada,mapa)

    if atributo:
        ax = litologia.plot('SIGLA')
        gdf.plot(atributo,ax=ax)
    else:
        ax = litologia.plot('SIGLA')
        gdf.plot(ax=ax)

In [131]:
def plot_brazil(gdf,atributo=None):
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    brazil = world[world.name == 'Brazil']
    if atributo:
        ax = brazil.boundary.plot(color='black',
                                  figsize=(12,12))
        gdf.plot(column=atributo,
                 ax=ax,
                 legend=True)
    else:
        ax = brazil.boundary.plot(color='black')
        gdf.plot(ax=ax)

In [8]:
# Abrindo os vetores descritivos dos projetos aerogeofisicos
proj_geof = importar_geologico('proj_aerogeof')

In [143]:
proj_geof_500m = proj_geof[proj_geof['ESPACAMENT'] == '500']

In [146]:
proj_geof_500m
proj_geof_500m['DISPONIBIL'].isnull().sum()

42

In [141]:
plt.plot(proj_geof_1000m)

ConversionError: Failed to convert value(s) to axis units: array(['Rondonópolis', 'Xingu-Araguaia', 'Rio Meia Ponte',
       'Franca (Área I)',
       'Rochas Básicas e Ultrabásicas de Vitória da Conqui',
       'Projeto Aerogeofísico Serra de Itiúba',
       'Projeto Aerogeofísico São Paulo - Rio de Janeiro (Área Rio de Janeiro)',
       'Projeto Aerogeofísico Extremo - Sudeste do Brasil',
       'Projeto Aerogeofísico Província Aurífera do Tapajós (Bloco I)',
       'Projeto Aerogeofísico Província Aurífera do Tapajós (Bloco II)',
       'Projeto Aerogeofísico São Paulo - Rio de Janeiro (Área São Paulo)',
       'Tocantínia', 'Esperança', 'Alto Garças', 'Surumu',
       'Projeto Aerogeofísico Serra do Mar Sul',
       'Ponta Grossa-Criciúma(Área I)', 'Araçuaí', 'Camaquã (Área II)',
       'Projeto Aerogeofísico Camaquã (Área I)', 'Barreiro',
       'Franca (Área II)',
       'Projeto Aerogeofísico Borda Leste da Bacia do Maranhão',
       'Projeto Aerogeofísico Rio Acaraú', 'Serra da

In [91]:
# Com este plot, podemos visualizar que a area de cobertura aerogeofisica e altissima, porem,
#  precisamos analisar melhor estes dados para que possa ser utilizados nos modelos preditivos.

plot_brazil(proj_geof,atributo='ESPACAMENT')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:
proj_geof.columns

Index(['ID_PROJETO', 'TITULO', 'METODOS', 'ALTURA_VOO', 'AREA_LEVAN',
       'QUILOMETRA', 'DIRECAO_LV', 'DIRECAO_LC', 'ESPACAMENT', 'ESPACAME_1',
       'ID_SERIE', 'SERIE', 'DISPONIBIL', 'IMAGENS', 'SHAPE_AREA', 'SHAPE_LEN',
       'geometry'],
      dtype='object')

In [35]:
#proj_geof.geometry.area.sum()

In [27]:
print(type(proj_geof['ESPACAMENT'][0]))
print(list(proj_geof.ESPACAMENT.unique()))

<class 'str'>
['2.5', '500', '1500', '1000', '3000', '2000', '3', '0', '250', '150', '200', '2500', '100', '4000', '2', '125', '400', '300', '1', '5', '4', '2,5', '6000', '8000', '6', '1,5', '7']


In [124]:
print(f"Foram identificados {len(proj_geof[proj_geof['ESPACAMENT'] == '500'])} projetos aerogeofisicos")
print(f"Lista de Titulos: {proj_geof[proj_geof['ESPACAMENT'] == '500']['TITULO']}")

Foram identificados 111 projetos aerogeofisicos
Lista de Titulos: 1      Projeto Aerogeofísico Escudo do Rio Grande do Sul
4           Projeto Aerogeofísico Sudeste do Mato Grosso
5              Projeto Aerogeofísico Sudeste de Rondônia
6               Projeto Aerogeofísicos Serra dos Apiacás
10     Projeto Aerogeofísico Reserva Nacional do Cobr...
                             ...                        
277                                 Serro (Área-Detalhe)
281                  Campo Alegre de Lourdes - Mortugaba
288               Projeto Aerogeofísico Pernambuco-Piauí
290                         Projeto Aerogeofísico Gurupi
295    Área 18 Minas Gerais - Porto Cajueiro - São Fr...
Name: TITULO, Length: 111, dtype: object


In [125]:
proj_geof_500m = proj_geof[proj_geof['ESPACAMENT'] == '500']

In [ ]:
# Abrindo os vetores litológicos utilizando a função definida acima
litologia_100k = importar_geologico('litologia_100k')
litologia_250k = importar_geologico('litologia_250k')

In [139]:
plot_brazil(proj_geof_500m.boundary)
plt.annotate(s='code',)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/tmp/ipykernel_8421/3947026308.py:2: MatplotlibDeprecationWarning: The 's' parameter of annotate() has been renamed 'text' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  plt.annotate(s='code')


TypeError: annotate() missing 1 required positional argument: 'xy'

In [137]:
#int(proj_geof['ESPACAMENT'])

In [87]:
proj_geof_500m = proj_geof[proj_geof.ESPACAMENT == '500']

In [128]:
#ax = litologia_100k.plot('SIGLA')
#proj_geof_500m.boundary.plot(ax=ax)

plot_base(proj_geof_500m.boundary,camada='litologia_100k')  # esta funçao e lenta pois importa toda a camada vetorial novamente e
                                                            # nao deve ser usada caso a litologia ja tenhasido definida

/home/ggrl/anaconda3/envs/geologist_bot/lib/python3.8/site-packages/geopandas/plotting.py:658: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(figsize=figsize)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Listando atributos geologicos das geometrias importadas.

In [ ]:
# Abrindo os vetores litológicos utilizando a função definida acima
litologia_100k = importar_geologico('litologia_100k')
litologia_250k = importar_geologico('litologia_250k')

In [40]:
# Abrindo os vetores litológicos de um unico mapa utilizando a função definida acima
varginha_100k = importar_geologico('litologia_100k','Varginha - UFRJ')

In [45]:
# atributos descritivos de cada uma das camadas litológicas (geometrias)
print("Lista de atributos descritivos classes litologicas: ")
print(list(varginha_100k.columns))

Lista de atributos descritivos classes litologicas: 
['ID_UNIDADE', 'SIGLA', 'HIERARQUIA', 'NOME', 'AMBIENTE_T', 'SUB_AMBIEN', 'SIGLA_PAI', 'NOME_PAI', 'LEGENDA', 'ESCALA', 'MAPA', 'LITOTIPOS', 'RANGE', 'IDADE_MIN', 'IDADE_MAX', 'EON_MIN', 'EON_MAX', 'ERA_MIN', 'ERA_MAX', 'SISTEMA_MI', 'SISTEMA_MA', 'EPOCA_MIN', 'EPOCA_MAX', 'SIGLAS_HIS', 'SHAPE_AREA', 'SHAPE_LEN', 'geometry']


In [46]:
# Plotando os vetores descritos pelos geólogos da cprm
varginha_100k.plot('SIGLA')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [47]:
print(varginha_100k.shape)
print(f"{varginha_100k.shape[0]} geometrias litologicas descritas em {varginha_100k.shape[1]} atributos")

(224, 27)
224 geometrias litologicas descritas em 27 atributos


In [48]:
# Observando o formato da descrição da coluna litotipos
varginha_100k_litotipos = varginha_100k.LITOTIPOS.unique()
print(f"{len(list(varginha_100k_litotipos))} litotipos reconhecidos")
list(varginha_100k_litotipos)

8 litotipos reconhecidos


['Diorito, Enderbito, Gabronorito, Gnaisse, Gnaisse granulitizado, Granulito, Granulito máfico, Mangerito, Olivina websterito, Ortognaisse tonalítico',
 'Biotita xisto, Gnaisse',
 None,
 'Albita anatexito, Biotita xisto, Gnaisse granulitizado, Gondito, Granulito máfico, Metamáfica, Rocha calcissilicática',
 'Anfibolito, Mármore, Metagrauvaca, Metaultramafito, Muscovita xisto, Paragnaisse, Quartzito, Quartzo xisto',
 'Muscovita-quartzo xisto, Quartzito',
 'Anfibolito, Biotita xisto, Gnaisse, Rocha calcissilicática',
 'Filito, Quartzito, Xisto']

In [49]:
# com estas funçoes utilizadas asssimas, podemos definir uma funçao que descreve o nosso dado vetorial
def describe_geologico(gdf):
    lista_colunas = list(gdf.columns)
    lista_shape = gdf.shape
    lista_litotipos = list(gdf.LITOTIPOS.unique())
    lista_legenda = list(gdf.LEGENDA.unique())

    dic_litologico = {'lista_colunas': lista_colunas,
                      'lista_shape': lista_shape,
                      'lista_litotipos': lista_litotipos,
                      'lista_legenda': lista_legenda}
    return dic_litologico   

In [50]:
dic_litoligico = describe_geologico(varginha_100k)

In [58]:
print(f"Numero de geometrias descritas na carta geologica: {len(dic_litoligico['lista_litotipos'])}")
print("Lista de valores unicos da coluna Litotipo: ")
list(dic_litoligico['lista_litotipos'])

Numero de geometrias descritas na carta geologica: 8
Lista de valores unicos da coluna Litotipo: 


['Diorito, Enderbito, Gabronorito, Gnaisse, Gnaisse granulitizado, Granulito, Granulito máfico, Mangerito, Olivina websterito, Ortognaisse tonalítico',
 'Biotita xisto, Gnaisse',
 None,
 'Albita anatexito, Biotita xisto, Gnaisse granulitizado, Gondito, Granulito máfico, Metamáfica, Rocha calcissilicática',
 'Anfibolito, Mármore, Metagrauvaca, Metaultramafito, Muscovita xisto, Paragnaisse, Quartzito, Quartzo xisto',
 'Muscovita-quartzo xisto, Quartzito',
 'Anfibolito, Biotita xisto, Gnaisse, Rocha calcissilicática',
 'Filito, Quartzito, Xisto']

In [57]:
print(f"Numero de geometrias descritas na carta geologica: {len(dic_litoligico['lista_legenda'])}")
print("Lista de valores unicos da coluna Legenda: ")
list(dic_litoligico['lista_legenda'])

Numero de geometrias descritas na carta geologica: 11
Lista de valores unicos da coluna Legenda: 


['Unidade granulítica basal',
 'Biotita xisto/gnaisse rico em plagioclássio, com granada, cianita e/ou silimanita, estaurolita, rutilo e muscovita.  Plagioclásio biotita xisto com muscovita, granada, estaurolita, cianita e sillimanita\n',
 'Depósitos aluvionares: Cascalho, areia e lama inconsolidada.',
 'Ortognaisse migmatíticos de composição monzo-graníticas a tonalítica com intercalções de anfibolitos de rochas ultramáficas.',
 'Cianita-granada granulitos a biotita, rutilo e ilmenita, Cianita K-feldspato gnaisse, intercalações de granulito máfico (plagioclásio-piroxênio-granada). \n intercalações de cianita-granada quartzito, gnaisses calcissilicáticos, anfibolito com ort',
 'Metagrauvaca, (muscovita)-biotita paragnaisse bandado, com intercalações de quartzito, muscovita-quartzo xisto, quartzitos, anfibolito e mármores, localmente migmatítico.',
 'Quartzitos com intercalações de muscovita-quartzo xistos.',
 'Biotita xisto-gnaisse grosso com intercalações de quartzito, cianita granada

## Reconhecendo as litologia mapeadas em suas escalas
- Faremos um filtro na coluna LEGENDA e LITOTIOPOS para selecionar as classes de interesse e, com estas geometrias, selecionaremos os mapas que as contém.
- Assim podemos identificar quais cartas geológicas e suas escalas podemos trabalhar.

In [113]:
def filtro(gdf,mineral):
    filtrado = gdf[gdf['LITOTIPOS'].str.contains(mineral)]
    if filtrado.empty:
        print(f"{list(gdf['LITOTIPOS'].unique())}")
    else:
        return filtrado

In [114]:
litologia_grafitosa_100k = filtro(litologia_100k,'graf')

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
litotipos_vazio = litologia_100k[litologia_100k.LITOTIPOS.isna()]
litotipos_vazio.LITOTIPOS

In [126]:
#Foi necessário inserir a str 'Vazio' em celas que não continham valores para que a função de filtro funcionasse corretamente na coluna LEGENDA.

#litologia_1kk.fillna('Vazio',inplace=True)
#litologia_250k.fillna('Vazio',inplace=True)
litologia_100k.fillna('Vazio',inplace=True)
#litologia_50k.fillna('Vazio',inplace=True)

In [110]:
Grafita_l1kk = filtro(litologia_1kk,'Graf')
print(list(Grafita_l1kk['LITOTIPOS'].unique()))

['Anfibolito, Grafita xisto, Metapelito, Quartzito', 'Formação ferrífera bandada, Grafita xisto, Metachert, Rocha metassedimentar, Rocha metavulcânica ultramáfica, Serpentinito', 'Biotita-anfibólio xisto, Biotita-quartzo xisto, Filito, Grafita xisto, Mármore calcítico, Mármore dolomítico, Metatufo, Quartzo-mica xisto, Rocha calcissilicática, Xisto carbonático', 'Clorita xisto, Grafita xisto, Hornblenda gnaisse, Metacalcário, Muscovita quartzito, Muscovita-biotita xisto, Quartzito ferruginoso, Talco xisto', 'Clorita xisto, Grafita xisto, Metachert, Metadacito, Metatufo, Xisto', 'Biotita gnaisse, Gnaisse, Gondito, Grafita xisto, Granada gnaisse, Kinzigito, Mármore, Muscovita quartzito, Quartzito, Rocha calcissilicática, Sericita xisto, Xisto', 'Grafita xisto, Metachert, Muscovita quartzito, Quartzito ferruginoso, Xisto, Xisto aluminoso', 'Clorita xisto, Cloritito, Formação ferrífera bandada, Grafita xisto, Muscovita-biotita xisto, Piroxenito, Serpentinito, Talco xisto, Xisto magnesiano',

In [28]:
Grafita_l1kk.plot('SIGLA')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [40]:
base = brazil.boundary.plot(color='black',figsize=(12,12))

leg_grafitosa_l1kk.plot(ax=base,facecolor='grey')

lit_grafitoso_1kk.plot(ax=base,facecolor='black')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [59]:
list_gr_100k=mapas_lito_gr_100k
indice=0
for i in mapas_lito_gr_100k:
    print(i)
    list_gr_100k[indice] = l_100k[l_100k['MAPA'].str.contains("i")]
    indice+=1


Carta geológica da folha Cerro Azul
Carta geológica da folha Apiaí
Carta geológica da folha Coromandel
Carta geológica da folha Serra da Tiririca
Carta geológica da folha Guarda Mor
Carta geológica da folha Cabeceira Grande
Carta geológica da folha Ribeirão Arrojado
Carta geológica da folha Alpinópolis - UERJ
Carta geológica da folha Patos de Minas
Carta geológica da folha Paracatu
Carta geológica da folha Arrenegado
Carta geológica da folha Piumhi - UFRJ
Carta geológica da folha Guapé - UERJ
Carta geológica da folha Cacimba
Carta geológica da folha Itararé
Carta geológica da folha Serra da Aldeia
Carta geológica da folha Brasilândia
Carta geológica da folha Monte Carmelo
Carta geológica da folha Lagamar
Carta geológica da folha Caiana
Carta geológica da folha Paranatinga
Carta geológica da folha Monte Azul - UFMG


In [3]:
len(list_gr_100k)


NameError: name 'list_gr_100k' is not defined

### Desafio desnecessário para provar meus conhecimentos com for loops:

In [540]:
lista_mapa = list(l_1kk.MAPA.unique())
print(len(lista_mapa))
lista_nome=[]

for mapa in lista_mapa:
    trash=0
    Mapa=[]
    for letra in mapa:
        if trash<25:
            trash+=1
        else:
            Mapa+=letra
    lista_nome += [''.join(Mapa)]

In [ ]:
lista_nome

In [510]:
folha=lista_nome
indice=0
for i in lista_nome:
    print(i)
    folha[indice] = l_1kk[l_1kk['MAPA'].str.contains(i)].dissolve('MAPA')
    indice+=1

Contamana
Javari
Rio Branco
Juruá
Içá
Pico da Neblina
Guaporé
Porto Velho
Purus
Manaus
Boa Vista
Corumbá
Cuiabá
Juruena
Tapajós
Santarém
Tumucumaque
Uruguaiana
Campo Grande
Asunción
Lagoa Mirim
Porto Alegre
Curitiba
Paranapanema
Goiânia
Goiás
Tocantins
Araguaia
Belém
Macapá
Iguape
Rio de Janeiro
Belo Horizonte
Brasília
Rio São Francisco
Teresina
São Luis
Vitória
Rio Doce
Salvador
Aracaju
Jaguaribe
Fortaleza
Recife
Natal


In [517]:
folha[31].plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

tired for now

In [544]:
lista_mapa = list(l_100k.MAPA.unique())
lista_mapa

['Carta geológica da folha Paulo Saldanha',
 'Carta geológica da folha Rio Tanaru - SD.20-X-B-IV',
 'Carta geológica da folha Roncador',
 'Carta geológica da folha Rio Pardo',
 'Carta geológica da folha Pimenta Bueno',
 'Carta geológica da folha Serra da Providência',
 'Carta geológica da folha Ji Paraná',
 'Carta geológica da folha Vila Nova',
 'Carta geológica da folha Vila de Tepequém',
 'Carta geológica da folha Betânia - UFMT',
 'Carta geológica da folha Rio Escondido',
 'Carta geológica da folha Ilha do Porto',
 'Carta geológica da folha Porto Triunfo SD.20-X-B-V',
 'Carta geológica da folha NA-20-Z-D-III',
 'Carta geológica da folha Novo Paraíso',
 'Carta geológica da folha Santa Rita - UFMT',
 'Carta geológica da folha Santa Barbara - UFMT',
 'Carta geológica da folha Serra da Borda',
 'Carta geológica da folha Rio Novo',
 'Carta geológica da folha Rio Pindaituba - UFMT',
 'Carta geológica da folha Vila Oeste',
 'Carta geológica da folha Jauru - UFMT',
 'Carta geológica da folh

In [546]:
lista_nome=[]
for mapa in lista_mapa:
    trash=0
    Mapa=[]
    for letra in mapa:
        if trash<25:
            trash+=1
        else:
            Mapa+=letra
    lista_nome += [''.join(Mapa)]

In [547]:
len(lista_nome)

311

In [549]:
folha=lista_nome
indice=0
for i in lista_nome:
    print(i)
    folha[indice] = l_100k[l_100k['MAPA'].str.contains(i)].dissolve('MAPA')
    indice+=1

Paulo Saldanha
Rio Tanaru - SD.20-X-B-IV
Roncador
Rio Pardo
Pimenta Bueno
Serra da Providência
Ji Paraná
Vila Nova
Vila de Tepequém
Betânia - UFMT
Rio Escondido
Ilha do Porto
Porto Triunfo SD.20-X-B-V
NA-20-Z-D-III
Novo Paraíso
Santa Rita - UFMT
Santa Barbara - UFMT
Serra da Borda
Rio Novo
Rio Pindaituba - UFMT
Vila Oeste
Jauru - UFMT
Rio Branco - UFMT
Ilha Porto Escondido
Colônia São Lourenço
Fazenda Santa Otília
Aldeia Tomázia
Rio Perdido
Fazenda Margarida
Vila Campão
São Domingos
Rio Ratão
Cuiú-Cuiú
Cachoeira Seca
Vila Planalto
Ilha Mambuai
Uruá
Barão de Melgaço - UFMT
Rio Nhandu
Jardim do Ouro
Igrejinha - UNISINOS
Bagé
Aceguá
Lagoa da Meia Lua
Brasilândia
Cacimba
Paranatinga
Rio Braço Sul
Serra do Cachimbo
Curral de Pedras
Arroio Grande
Jaguarão
Passo São Diogo
Hulha Negra
Pinheiro Machado
Lavras do Sul
Santa Maria - RS
Sobradinho
Agudo
Três Passos
Frederico Westphalen
Matarazzo
Pelotas
Pedro Osório
Placa Nativa
Nova Xavantina
Rio Bacajá - UFPR
Gravataí - UFRGS
Sanclerlândia
Rio In

In [550]:
len(folha)

311